In [ ]:
#!python --version

In [ ]:
#from __future__ import print_function
import numpy as np 
from sklearn import tree
from sklearn import svm
from sklearn.model_selection import cross_val_score
import math 
from sklearn.cluster import KMeans, DBSCAN

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.

In [ ]:
# !pip uninstall pandas -y

In [ ]:
# !pip install pandas==0.25.1

In [ ]:
import pandas as pd
pd.__version__

'0.25.1'

In [ ]:
# !pip uninstall scikit-learn -y

In [ ]:
# !pip install scikit-learn==0.21.2

In [ ]:
import sklearn
sklearn.__version__

'0.21.2'

In [ ]:
import pickle

In [ ]:
import datetime 

def dateTimeFunction(date, time):
  month,day,year = dateFunc(date)
  hour,minute,second = timeFunc(time)
  dt = datetime.datetime(int(year),int(month),int(day),int(hour),int(minute),int(second))
  return dt

def dateFunc(date):
  if '/' in date:
    m,d,y = date.split('/')
    return m,d,y
  elif '-' in date:
    d = date.split(' ')
    y,m,d = d[0].split('-')
    return m,d,y
  else:
    #print('malfunction')
    return None 

def timeFunc(time):
  h,m,s = time.split(':')
  return h,m,s

In [ ]:
def isNaN(num):
    return num != num

In [ ]:
# BWZ Carb Input (grams) == 24

In [ ]:

def part1_meal_noMeal(df):
  meal_time = []
  no_meal_time = []
  carbs = []
  carb = None

  foundMeal = False

  mealCounter = 0
  noMealCounter = 0

  tempMealTime = 0
  tempNoMealTime = 0

  for index,row in df.iterrows():

    
    if isNaN(row[24]) == False and float(row[24]) > 0.0 and foundMeal == False:
      foundMeal = True
      tempMealTime = dateTimeFunction(row[1], row[2])
      noMealCounter = 0
      carb = row[24]
      
    elif isNaN(row[24]) == False and float(row[24]) > 0.0 and foundMeal == True: # rule 2
      tempMealTime = dateTimeFunction(row[1], row[2]) 
      mealCounter = 0
    

    if foundMeal == True:
      mealCounter += 1
    elif foundMeal == False:
      if noMealCounter == 0:
        tempNoMealTime = dateTimeFunction(row[1], row[2])
      noMealCounter += 1


    if noMealCounter == 24:
      no_meal_time.append(tempNoMealTime)
      tempNoMealTime = 0
      noMealCounter = 0

    if mealCounter == 24 and isNaN(row[24]) == False and float(row[24]) > 0.0: # rule 3
      tempMealTime = dateTimeFunction(row[1], row[2])
      meal_time.append(tempMealTime)
      carbs.append(carb)
      carb = None
      tempMealTime = 0
      foundMeal = False
      mealCounter = 0
    elif mealCounter == 24: # rule 1
      meal_time.append(tempMealTime)
      carbs.append(carb)
      carb = None
      tempMealTime = 0
      foundMeal = False
      mealCounter = 0

  return meal_time, no_meal_time, carbs


In [ ]:

def part1_extract_p_q(df, meal, noMeal):

  # ensure first time for meal and no meal is not before start of cgm data
  elimCount_meal = 0
  elimCount_noMeal = 0
  for i,r in df.iterrows():
    dt = dateTimeFunction(r[1],r[2])

    mdt = meal[elimCount_meal] 
    nmdt = noMeal[elimCount_noMeal]

    if mdt < dt:
      elimCount_meal += 1
    
    if nmdt < dt:
      elimCount_noMeal += 1

    if mdt > dt and nmdt > dt:
      break
  
  meal = meal[elimCount_meal:]
  noMeal = noMeal[elimCount_noMeal:]

  p = []
  q = []
  #meal_carbs = []

  mealIndex = 0
  noMealIndex = 0

  indexCounter = 0

  mealDone = False
  noMealDone = False

  for index, row in df.iterrows():

    mealDateTime = meal[mealIndex]
    noMealDateTime = noMeal[noMealIndex]

    dateTime = dateTimeFunction(row[1], row[2])

    if mealDateTime < noMealDateTime and mealDone == False: 
      if indexCounter > len(df) - 24:
        mealDone = True
      elif dateTime > mealDateTime and mealIndex < len(meal) - 1:
        arr = df['Sensor Glucose (mg/dL)'][indexCounter-6:indexCounter+24]
        p.append(arr)
        #meal_carbs.append(df['BWZ Carb Input (grams)'][indexCounter])
        mealIndex += 1
      elif dateTime > mealDateTime and mealIndex == len(meal) - 1:
        arr = df['Sensor Glucose (mg/dL)'][indexCounter-6:indexCounter+24]
        p.append(arr)
        #meal_carbs.append(df['BWZ Carb Input (grams)'][indexCounter])
        mealDone = True


    elif mealDateTime > noMealDateTime and noMealDone == False:
      if indexCounter > len(df) - 24:
        noMealDone = True
      elif dateTime > noMealDateTime and noMealIndex < len(noMeal) - 1:
        arr = df['Sensor Glucose (mg/dL)'][indexCounter:indexCounter+24]
        q.append(arr)
        noMealIndex += 1
      elif dateTime > noMealDateTime and noMealIndex == len(noMeal) - 1:
        arr = df['Sensor Glucose (mg/dL)'][indexCounter:indexCounter+24]
        q.append(arr)
        noMealDone = True      

    indexCounter += 1
  return p, q #, meal_carbs

In [ ]:
def handle_missing_data(p,q, carbs):
  new_p = []
  new_q = []
  new_carbs = []
  x = 0
  for i in p:
    drop = False
    for j in i:
      if isNaN(j):
        drop = True
    if drop == False:
      new_p.append(i)
      new_carbs.append(carbs[x])
    x += 1
  
  for i in q:
    drop = False
    for j in i:
      if isNaN(j):
        drop = True
    if drop == False:
      new_q.append(i)

  return new_p, new_q, new_carbs

In [ ]:

# Part 2 : Feature Extraction

# feature 1
def feature0(x):
  cgm_min = min(x)
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  tdiff = max_i - min_i
  return tdiff 

def feature00(x):
  cgm_min = x[0]
  cgm_max = max(x)
  min_i = 0 #x.index(cgm_min)
  max_i = x.index(cgm_max)
  tdiff = max_i - min_i
  return tdiff

def feature000(x):
  cgm_min = x[6]
  cgm_max = max(x)
  min_i = 6 #x.index(cgm_min)
  max_i = x.index(cgm_max)
  tdiff = max_i - min_i
  return tdiff 

# feature 1
def feature1(x):
  cgm_min = min(x)
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  tdiff = max_i - min_i
  td_in_min = tdiff * 5
  return td_in_min 

def featureA(x):
  cgm_min = x[0]
  cgm_max = max(x)
  min_i = 0 #x.index(cgm_min)
  max_i = x.index(cgm_max)
  tdiff = max_i - min_i
  td_in_min = tdiff * 5
  return td_in_min

def featureAZ(x):
  cgm_min = x[6]
  cgm_max = max(x)
  min_i = 6 #x.index(cgm_min)
  max_i = x.index(cgm_max)
  tdiff = max_i - min_i
  td_in_min = tdiff * 5
  return td_in_min 

# feature 2
def feature2(x):
  cgm_min = min(x)
  cgm_max = max(x)
  cg = cgm_max - cgm_min
  cgN = cg / cgm_min
  return cgN 

def featureB(x):
  cgm_min = x[0]
  cgm_max = max(x)
  cg = cgm_max - cgm_min
  cgN = cg / cgm_min
  return cgN 

def featureBZ(x):
  cgm_min = x[6]
  cgm_max = max(x)
  cg = cgm_max - cgm_min
  cgN = cg / cgm_min
  return cgN 
  
# feature 3,4,5,6: sinusoid frequency response ~ fast fourier transform : Pf1 , f1, Pf2, f2
def feature_sfr_fft(x):
  retval = []
  fft = np.fft.fft(np.array(x))
  for i in fft[:24]:
    real = i.real
    imag = i.imag 
    retval.append(real)
    retval.append(imag) 
  return retval

# feature 7: slope of cgm diff over time from meal start to cgm max
def feature7(x):
  cgm_min = min(x)
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return 0
  dt_in_min = dt * 5
  slope = cg / dt_in_min 
  return slope 

def featureC(x):
  cgm_min = x[0]
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return 0
  dt_in_min = dt * 5
  slope = cg / dt_in_min 
  return slope

def featureCZ(x):
  cgm_min = x[6]
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return 0
  dt_in_min = dt * 5
  slope = cg / dt_in_min 
  return slope 

def feature70(x):
  cgm_min = min(x)
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return 0
  # dt_in_min = dt * 5
  slope = cg / dt 
  return slope 

def featureC0(x):
  cgm_min = x[0]
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return 0
  # dt_in_min = dt * 5
  slope = cg / dt 
  return slope

def featureCZ0(x):
  cgm_min = x[6]
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return 0
  # dt_in_min = dt * 5
  slope = cg / dt 
  return slope 

def feature80(x):
  cgm_min = min(x)
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return
  # dt_in_min = dt 
  slope = cg / dt 
  return slope / dt

def featureD0(x):
  cgm_min = x[0]
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return
  # dt_in_min = dt * 5
  slope = cg / dt 
  return slope / dt

def featureDZ0(x):
  cgm_min = x[6]
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return
  # dt_in_min = dt 
  slope = cg / dt 
  return slope / dt


def feature8(x):
  cgm_min = min(x)
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return
  dt_in_min = dt * 5
  slope = cg / dt_in_min 
  return slope / dt_in_min

def featureD(x):
  cgm_min = x[0]
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return
  dt_in_min = dt * 5
  slope = cg / dt_in_min 
  return slope / dt_in_min

def featureDZ(x):
  cgm_min = x[6]
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return
  dt_in_min = dt * 5
  slope = cg / dt_in_min 
  return slope / dt_in_min


def feature9(x):
  cgm_min = min(x)
  cgm_max = max(x)
  cg = cgm_max - cgm_min
  return cg

def featureE(x):
  cgm_min = x[0]
  cgm_max = max(x)
  cg = cgm_max - cgm_min
  return cg

def featureEZ(x):
  cgm_min = x[6]
  cgm_max = max(x)
  cg = cgm_max - cgm_min
  return cg


def feature10(x):
  return max(x)

def feature11(x):
  return min(x)



In [ ]:
def part2_feature_extraction(npq):
  fpq = []

  for i in npq: 
    f = []

    f1 = feature1(i)
    f.append(f1)

    f2 = feature2(i)
    f.append(f2)



    f7 = feature7(i)
    f.append(f7)

    f8 = feature8(i)
    f.append(f8)

    f9 = feature9(i) 
    f.append(f9) 



    # fA = featureA(i)
    # f.append(fA)

    # fB = featureB(i)
    # f.append(fB)

    # fC = featureC(i)
    # f.append(fC)

    # fD = featureD(i)
    # f.append(fD) 

    # fE = featureE(i)
    # f.append(fE)

    # fAZ = featureAZ(i)
    # f.append(fAZ)

    # fBZ = featureBZ(i)
    # f.append(fBZ)

    # fCZ = featureCZ(i)
    # f.append(fCZ)

    # fDZ = featureDZ(i)
    # f.append(fDZ) 

    # fEZ = featureEZ(i)
    # f.append(fEZ)

    # f0 = feature0(i)
    # f.append(f0)
    
    # f00 = feature00(i)
    # f.append(f00)

    # f000 = feature000(i)
    # f.append(f000)




    fpq.append(f)
  
  return fpq

In [ ]:
#def main():

# read in data
df_insulin = pd.read_csv('InsulinData.csv', low_memory=False)
df_cgm = pd.read_csv('CGMData.csv', low_memory=False)

In [ ]:
# reverse order of dataframe so time is in chronological order
df_insulin = df_insulin.iloc[::-1]
df_cgm = df_cgm.iloc[::-1]

In [ ]:
# Part 1 
meal, noMeal, carbs = part1_meal_noMeal(df_insulin)
p, q = part1_extract_p_q(df_cgm, meal, noMeal)

In [ ]:
# handle missing data
new_p, new_q, carbs = handle_missing_data(p,q, carbs)


In [ ]:
# Part 2
np_ = [list(i) for i in new_p]

features  = part2_feature_extraction(np_)

In [ ]:

def numberBins(carbs):
  mi = min(carbs)
  ma = max(carbs)
  diff = ma - mi
  #print(diff)
  numBins = math.ceil(diff/20)
  return numBins 

def labelMaker(carbs, numBins):
  labels = []
  for carb in carbs:
    label = math.floor(carb / 20) % numBins
    labels.append(label)
  return np.array(labels)

def removeFeatureWithNanCarb(features, carbs):
  new_features = []
  new_carbs = []

  for i in range(len(carbs)):
    
    if isNaN(carbs[i]) == False:
      new_features.append(features[i])
      new_carbs.append(carbs[i])
  return new_features, new_carbs


In [ ]:
features, carbs = removeFeatureWithNanCarb(features,carbs)
numBins = numberBins(carbs)
labels = labelMaker(carbs, numBins)

In [ ]:
# project 3 k-means

In [ ]:
df_mf = pd.DataFrame(features)

In [ ]:
kmeans = KMeans(n_clusters=numBins)

In [ ]:
predicted = kmeans.fit_predict(df_mf)

In [ ]:
kmeans_sse = np.sum((labels - predicted)**2)

In [ ]:
print(kmeans_sse)

AttributeError: ignored

In [ ]:
def entropy(labels, predicted, numBins):
  entropy = []
  total_ci = [0] * numBins
  for i in range(numBins):
    entropy.append([0] * numBins)
  for i in range(len(labels)):
    p = predicted[i]
    l = labels[i] 
    total_ci[l] += 1 
    entropy[p][l] = entropy[int(p)][int(l)] + 1
  for i in range(numBins):
    for j in range(numBins):
      e = entropy[i][j] / total_ci[j]
      mlog = 0
      if e == 0:
        mlog = 0
      else: 
        mlog = math.log(e, numBins)
      entropy[i][j] = -1 * e * mlog
  return np.sum(np.array([total_ci[i] * np.sum(np.array(entropy[i])) for i in range(len(entropy))])) / sum(total_ci)

In [ ]:
kmeans_entropy = entropy(labels, predicted, numBins)
print(kmeans_entropy)

0.8730921703433953


In [ ]:
# def purity(labels, predicted, numBins):
#   entropy = []
#   total_ci = [0] * numBins
#   total_ji = [0] * numBins
#   for i in range(numBins):
#     entropy.append([0] * numBins)
#   for i in range(len(labels)):
#     p = predicted[i]
#     l = labels[i] 
#     total_ci[l] += 1 
#     total_ji[p] += 1
#     entropy[p][l] = entropy[int(p)][int(l)] + 1
#   for i in range(numBins):
#     for j in range(numBins):
#       e = entropy[i][j] / total_ci[j]
#       if total_ji[j] == 0:
#         e = 0
#       else:
#         e = e / total_ji[j]
#       mlog = 0
#       if e == 0:
#         mlog = 0
#       else: 
#         mlog = math.log(e) 
#         if total_ji[j] == 0:
#           mlog = 0
#         else:
#           mlog = mlog / total_ji[j]
#       entropy[i][j] = -1 * e * mlog
#   return np.sum(np.array([total_ci[i] * np.sum(np.array(entropy[i])) for i in range(len(entropy))])) / sum(total_ci)

In [ ]:
kmeans_purity = 1 - kmeans_entropy#purity(labels, predicted, numBins)
print(kmeans_purity)

0.12690782965660474


In [ ]:
dbs_clustering = DBSCAN(eps=60, min_samples=1).fit_predict(df_mf)
print(dbs_clustering)
print(len(dbs_clustering))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
407


In [ ]:
for i in range(len(dbs_clustering)):
  if dbs_clustering[i] == -1:
    dbs_clustering[i] = 0
  elif dbs_clustering[i] >= numBins:
    dbs_clustering[i] = dbs_clustering[i] % numBins

In [ ]:
print(dbs_clustering)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
dbscan_sse = np.sum((labels - dbs_clustering)**2)
print(dbscan_sse)

dbscan_entropy = entropy(labels, dbs_clustering, numBins)
print(dbscan_entropy)

dbscan_purity= 1 - dbscan_entropy #purity(labels, dbs_clustering, numBins)
print(dbscan_purity)

1666
0.007557154996739249
0.9924428450032607


In [ ]:
predictions = [[kmeans_sse, dbscan_sse, kmeans_entropy, 0, kmeans_purity, 1]]

In [ ]:
results_df = pd.DataFrame(predictions)
results_df.to_csv('Result.csv', header=False, index=False)

In [ ]:
print(predictions)

[[2385, 1666, 0.8730921703433953, 0, 0.12690782965660474, 1]]
